In [23]:
!pip install nltk
!pip install spacy
!pip install jieba
!pip install transformers
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 79.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 94.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 36.6 MB/s eta 0:00:00


In [25]:
import nltk
import spacy
from jieba import posseg
import jieba
from collections import Counter
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nlp_en = spacy.load("en_core_web_sm")

from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt", src_lang="en_XX")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [9]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [35]:
class Reduction:

    def __init__(self,path,save_path):
        with open(path) as f:
          text = f.readlines()
        self.en_li = []
        self.zh_li = []
        for i, sent in enumerate(text):
            if i%2 == 0:
                self.en_li.append(sent.strip())
            elif i%2 == 1:
                self.zh_li.append(sent.strip())
        self.save_path = save_path


        


    def is_anticipatory_it(self):
        e = 1
        for en,zh in zip(self.en_li, self.zh_li):
            words = nltk.word_tokenize(en)
            tagged_words = nltk.pos_tag(words)
            
            for i, (word, pos) in enumerate(tagged_words):
                if word.lower() == "it" and pos == "PRP":
                    if i < len(tagged_words) - 1 and tagged_words[i + 1][1] == "VBZ" and '它' not in zh and '其' not in zh and '这' not in zh:
                        print(en)
                        print(zh)
                        with open(save_path,'a+') as r:
                            r.write(en+'\n'+zh+'\n'+"it:"+'\n')


    def removal_of_copula(self):
        e = 1
        for en,zh in zip(self.en_li, self.zh_li):
            words = nltk.word_tokenize(en)
            tagged_words = nltk.pos_tag(words)
            copulas = ['am', 'is', 'are', 'was', 'were', 'be', 'been']
            for word, pos in tagged_words:
                if word.lower() in copulas and '是' not in zh:
                    print(en)
                    print(zh)
                    print('-----------')
                    with open(save_path,'a+') as r:
                        r.write(en+'\n'+zh+'\n'+word+':'+'\n')


    def removal_of_copula(self):
        e = 1
        for en,zh in zip(self.en_li, self.zh_li):
            words = nltk.word_tokenize(en)
            tagged_words = nltk.pos_tag(words)
            copulas = ['am', 'is', 'are', 'was', 'were', 'be', 'been']
            for word, pos in tagged_words:
                if word.lower() in copulas and '是' not in zh:
                    print(en)
                    print(zh)
                    print('-----------')
                    with open(save_path,'a+') as r:
                        r.write(en+'\n'+zh+'\n'+word+':'+'\n')

    def removal_of_pronoun(self):
        e = 1
        for en,zh in zip(self.en_li, self.zh_li):
            words = nltk.word_tokenize(en)
            tagged_words = nltk.pos_tag(words)
            pronouns = [word.lower() for word, pos in tagged_words if pos == 'PRP' or pos == 'PRP$']
            pronoun_counts = Counter(pronouns)

            repeated_pronouns = [pronoun for pronoun, count in pronoun_counts.items() if count > 1]
            if repeated_pronouns != [] and len(repeated_pronouns) == 1 and 'exucse' not in en and 'Excuse' not in en:

                en_token = tokenizer(repeated_pronouns[0], return_tensors="pt")
                generated_tokens = model.generate(**en_token, forced_bos_token_id=tokenizer.lang_code_to_id["zh_CN"])
                zh_text_trans = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]
                if '的' in zh_text_trans:
                    zh_tok_trans = zh_text_trans.replace('的','')
                else:
                    zh_tok_trans = zh_text_trans
                
                    
                zh_tok_li = [zh_tok for zh_tok in jieba.cut(zh)]
                pronoun_counts_zh = Counter(zh_tok_li)
                # print(pronoun_counts_zh)
                if zh_tok_trans in pronoun_counts_zh and pronoun_counts_zh[zh_tok_trans] <= 1:
                        print(en)
                        print(zh)
                        print(repeated_pronouns)
                        print(zh_tok_trans)
                        print('----------------')
                        with open(save_path,'a+') as r:
                            r.write(en+'\n'+zh+'\n'+repeated_pronouns[0]+':'+'\n')
                        
                  
    def removal_of_determiner(self):
        e = 1
        for en,zh in zip(self.en_li, self.zh_li):
            words = nltk.word_tokenize(en)
            tagged_words = nltk.pos_tag(words)

            possessive_pronouns = [word.lower() for word, pos in tagged_words if pos == 'PRP$']
            if len(possessive_pronouns) == 1:
                en_token = tokenizer(possessive_pronouns[0], return_tensors="pt")
                generated_tokens = model.generate(**en_token, forced_bos_token_id=tokenizer.lang_code_to_id["zh_CN"])
                zh_text_trans = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]

                if '的' in zh_text_trans:
                    zh_tok_trans = zh_text_trans.replace('的','')
                else:
                    zh_tok_trans = zh_text_trans
                
                if zh_tok_trans not in zh:
                    print(en)
                    print(zh)
                    print(possessive_pronouns[0])




    def removal_of_preposition(self):
        e = 1
        for en,zh in zip(self.en_li, self.zh_li):
            words = nltk.word_tokenize(sentence)
            tagged_words = nltk.pos_tag(words)

            prepositions = [word.lower() for word, pos in tagged_words if pos == 'IN']
            if len(prepositions) == 1:
                en_token = tokenizer(prepositions[0], return_tensors="pt")
                generated_tokens = model.generate(**en_token, forced_bos_token_id=tokenizer.lang_code_to_id["zh_CN"])
                zh_text_trans = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]
                print(zh_text_trans)


                


            

spath = '/content/gdrive/MyDrive/Colab Notebooks/literal translationese/dataset/Bilingual/Education/Bi-Education.txt'
save_path = '/content/gdrive/MyDrive/Colab Notebooks/literal translationese/dataset/feature89_reduction_explicitation/removal_of_pronoun.txt'             
        
ptcl = Reduction(spath, save_path)

ptcl.removal_of_pronoun()




/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (200) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


流式输出内容被截断，只能显示最后 5000 行内容。
After baptism Jesus came up out of the water at once, and at that moment heaven opened; he saw the Spirit of God descending like a dove to alight upon him; and a voice from heaven was heard saying, 'This is my Son,my Beloved, on whom my favor rests. '
耶稣受洗后，马上从水中出来；这时，天忽然开了。他看到圣灵象鸽子一样从天而降，落在了他身上。天上传下一个声音来说：“这就是我所宠爱的爱子。”
['my']
我
----------------
After breakfast, I directly undress all of my clothes, wrapped me with the cotton quilt, just stretched out two hands from the broken holes of my quilt, and hold a book, leaned against the wall on which mud had been dropped.
早饭后，我干脆把衣裳全部脱光，用棉花网套把自己包了起来，仅从网套的破洞里伸出两只手，捧着本书，靠在泥土剥落的墙上。
['my']
我
----------------
After dinner, as Tun-weng was leaving the table, he said, "He's chosen a good way.
晚饭吃完，遯翁出坐时，又说：“他这个办法很好。
['he']
他
----------------
After dinner, he decided that since he had been inactive all day, he would take a moonlight stroll.Then Miss Su called once more to ask if he felt better and if he would care to go to

KeyboardInterrupt: ignored